In [13]:
import os
import argparse
import torch
from torch.utils.data import DataLoader, IterableDataset
from torch.utils.data import get_worker_info
import torch.distributed as dist
import transformers
from transformers import default_data_collator
from transformers import AutoTokenizer
from omegaconf import DictConfig, OmegaConf as om
from tqdm import tqdm
import numpy as np
import random
random.seed(111)
from collections.abc import Mapping
from typing import Optional, Tuple, Union, Any, Dict, List
import logging		


# Add ModernBERT to path
# TODO: add morerngena distr target path argparse
import sys
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("."))

from src.text_data import NoStreamingGenomeDataset

class _GenomeDatasetForMasking(NoStreamingGenomeDataset):
	def __getitem__(self, index: int):
		shard_id, shard_sample_id = self.spanner[index]
		shard = self.shards[shard_id]
		sample = shard[shard_sample_id]
		return sample['file_id'], sample['line_id'], shard_id


data_dir= "/mnt/nfs_dna/shadskiy/promoters/pretrena/mds_v2/"
tokenizer = AutoTokenizer.from_pretrained("AIRI-Institute/gena-lm-bert-base-t2t")
# Create dataset
ds = _GenomeDatasetForMasking(
	local=data_dir,
	split="train",
	max_seq_len=1024,
	tokenizer = tokenizer,
)


/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
genomes = set()
last_shard=30
for i in tqdm(range(0,len(ds),1)):
	_ = ds[i]
	genome = _[0]
	shard_id = _[2]
	if shard_id>last_shard:
		print (i,shard_id)
		break
	else:
		if i % 5000 == 0:
			print (i,shard_id)
	# if genome.find("GCF_000001405")!=-1:
	# 	print (i)
	# 	print (genome)
	# 	break
	# else:
	# 	genomes.add(genome)
print (len(genomes))

  0%|          | 41/13525034 [00:00<18:48:31, 199.75it/s]

0 0


  0%|          | 5027/13525034 [00:24<18:36:13, 201.87it/s]

5000 1


  0%|          | 10029/13525034 [00:48<17:57:54, 208.97it/s]

...

50000 19


  0%|          | 55030/13525034 [04:26<17:46:17, 210.54it/s]

55000 21


  0%|          | 56205/13525034 [04:31<18:05:35, 206.78it/s]


KeyboardInterrupt: 

In [28]:
print (ds.spanner.shard_bounds[0:5])

[    0  2586  5093  7654 10256]


In [27]:
len(ds.shards[0])

2586